## Stappenplan en verdeling

1. aanmaken github
2. data preprocessing (verwijder duplicates and NaN)
3. bepaal de relevante features van de descriptors via een PCA
4. wat wordt test en wat wordt train
5. aanmaken model voor het maken van voorspellingen
    5.1 uitproberen welke methodes er toepasbaar zijn (random forest, k-NN)
    5.2 bepalen welke methode het beste is
6. verslag en presentatie 

1. Richard
2. Luuk, Richard
3. Robin, Tijn, Rik & Yael
4. 
5. Julien & Branco
6. 

## 5.1 Onderzoek over random forest methode
The random forest algorithm is an extension of the bagging method as it utilizes both bagging and feature randomness to create an uncorrelated forest of decision trees. While decision trees consider all the possible feature splits, random forests only select a subset of those features.

Random forest algorithms have three main hyperparameters, which need to be set before training. These include node size, the number of trees, and the number of features sampled. From there, the random forest classifier can be used to solve for regression or classification problems.

> Dit betekent dat alle descriptors onze features zijn waarvan per tree een random keuze wordt gemaakt. 
> The nodesize parameter specifies the minimum number of observations in a terminal node. Setting it lower leads to trees with a larger depth which means that more splits are performed until the terminal nodes.

The random forest algorithm is made up of a collection of decision trees, and each tree in the ensemble is comprised of a data sample drawn from a training set with replacement, called the bootstrap sample. Of that training sample, one-third of it is set aside as test data, known as the out-of-bag (oob) sample, which we’ll come back to later. Another instance of randomness is then injected through feature bagging, adding more diversity to the dataset and reducing the correlation among decision trees. Depending on the type of problem, the determination of the prediction will vary.

> Dit geeft dus aan dat onze verschillende moleculen niet allemaal tegelijk moeten worden gebruikt maar in secties. De grote van deze sectie is denk ik voor onszelf te bepalen. 

In [24]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris


## Test/train with equal ration 0 and 1

## try some of our models on the actual test set

# finger prints

In [25]:

data = pd.read_csv('cleaned_fingerprint_data', index_col=0)

X = data.drop(columns=['ERK2_inhibition', 'PKM2_inhibition']).copy()
y = data['ERK2_inhibition']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Bereken de ratio tussen 0 and 1 
ratio_train = y_train.value_counts(normalize=True)

# Bereken de ratio tussen 0 and 1
ratio_test = y_test.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the training set:")
print(ratio_train)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test)

# Weigth
# class_weights = {0: 1, 1: 1}  # Aanpassen op weigth distribution
rf_equal = RandomForestClassifier(n_estimators=1100)#, class_weight=class_weights)


rf_equal.fit(X_train, y_train)


y_pred = rf_equal.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nAccuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)
print(y_pred)

Ratio of 0 and 1 inhibitors in the training set:
0    0.954036
1    0.045964
Name: ERK2_inhibition, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
0    0.955357
1    0.044643
Name: ERK2_inhibition, dtype: float64

Accuracy: 0.9553571428571429
Confusion Matrix:
[[214   0]
 [ 10   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       214
           1       0.00      0.00      0.00        10

    accuracy                           0.96       224
   macro avg       0.48      0.50      0.49       224
weighted avg       0.91      0.96      0.93       224

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
untested_molecules=pd.read_csv('cleaned_fingerprint_data',index_col=0)
untested_molecules.head()



,Bit_696,Bit_1200,Bit_936,Bit_1089,Bit_1918,Bit_379,Bit_842,Bit_1153,Bit_876,Bit_927,...,Bit_1454,Bit_675,Bit_1921,Bit_388,Bit_1565,Bit_1100,Bit_393,Bit_492,ERK2_inhibition,PKM2_inhibition
C=C(C)c1nc(N)nc(N)n1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
C=CCNC(=O)CCCC(=O)NCC=C,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21,1,1,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
C=CCn1cc(Cl)c(=O)n(CC=C)c1=O,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X_prediction=untested_molecules.drop(columns=['ERK2_inhibition','PKM2_inhibition']).copy()
#try with the random forest where equal values train, test split was used
ERK_prediction_f=rf_equal.predict(X_prediction)
amount_positive=pd.DataFrame(ERK_prediction_f).loc[ERK_prediction_f==1]
len(amount_positive)

41

# variables

In [28]:
data = pd.read_csv('cleaned_descriptor_data', index_col=0)

X = data.drop(columns=['ERK2_inhibition', 'PKM2_inhibition']).copy()
y = data['ERK2_inhibition']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Bereken de ratio tussen 0 and 1 
ratio_train = y_train.value_counts(normalize=True)

# Bereken de ratio tussen 0 and 1
ratio_test = y_test.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the training set:")
print(ratio_train)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test)

# Weigth
# class_weights = {0: 1, 1: 1}  # Aanpassen op weigth distribution
rf_equal = RandomForestClassifier(n_estimators=1100)#, class_weight=class_weights)


rf_equal.fit(X_train, y_train)


y_pred = rf_equal.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nAccuracy: {accuracy}")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(report)
print(y_pred)

Ratio of 0 and 1 inhibitors in the training set:
0    0.954036
1    0.045964
Name: ERK2_inhibition, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
0    0.955357
1    0.044643
Name: ERK2_inhibition, dtype: float64

Accuracy: 0.9553571428571429
Confusion Matrix:
[[214   0]
 [ 10   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       214
           1       0.00      0.00      0.00        10

    accuracy                           0.96       224
   macro avg       0.48      0.50      0.49       224
weighted avg       0.91      0.96      0.93       224

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
untested_molecules=pd.read_csv('cleaned_descriptor_data',index_col=0)
untested_molecules.head()

,BertzCT,ExactMolWt,Kappa1,NOCount,BCUT2D_LOGPLOW,NumValenceElectrons,HeavyAtomCount,Chi2n,PEOE_VSA1,Kappa2,...,SMR_VSA7,NumAromaticCarbocycles,fr_benzene,BCUT2D_MRLOW,EState_VSA3,NumAromaticRings,Kappa3,PEOE_VSA2,ERK2_inhibition,PKM2_inhibition
SMILES,,,,,,,,,,,,,,,,,,,,,
C=C(C)c1nc(N)nc(N)n1,272.774159,151.085795,7.456952,5,-2.047656,58,11,2.155745,11.467335,2.531038,...,12.403340,0,0,1.044248,11.397509,1,1.635047,0.000000,0,0
C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1,628.087447,250.039672,11.736876,3,-1.991714,88,17,3.712350,9.154014,4.634332,...,51.860265,1,1,0.359025,16.364818,2,2.683837,0.000000,0,0
C=CCNC(=O)CCCC(=O)NCC=C,212.742421,210.136828,13.420000,4,-2.250322,84,15,3.039058,10.633577,8.993915,...,25.309912,0,0,-0.121740,32.351978,0,8.742061,9.589074,0,0
C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21,631.418725,232.084792,11.193326,5,-2.432024,88,17,3.552474,10.044842,4.288212,...,58.175409,1,1,-0.585723,15.763809,2,1.753868,4.794537,0,0
C=CCn1cc(Cl)c(=O)n(CC=C)c1=O,499.124344,226.050905,11.524349,4,-2.388420,80,15,2.905081,0.000000,4.794989,...,57.367473,0,0,0.625699,6.544756,1,2.274281,13.928736,0,0


In [30]:
X_prediction=untested_molecules.drop(columns=['ERK2_inhibition','PKM2_inhibition']).copy()
#try with the random forest where equal values train, test split was used
ERK_prediction_v=rf_equal.predict(X_prediction)
amount_positive=pd.DataFrame(ERK_prediction_v).loc[ERK_prediction_v==1]
len(amount_positive)

41

# intersect

In [62]:

finger_prediction=pd.DataFrame(ERK_prediction_f,index=X.index)
descriptor_prediction=pd.DataFrame(ERK_prediction_v,index=X.index)
data[(descriptor_prediction[0]==1) & (finger_prediction[0]==1)]

,BertzCT,ExactMolWt,Kappa1,NOCount,BCUT2D_LOGPLOW,NumValenceElectrons,HeavyAtomCount,Chi2n,PEOE_VSA1,Kappa2,...,SMR_VSA7,NumAromaticCarbocycles,fr_benzene,BCUT2D_MRLOW,EState_VSA3,NumAromaticRings,Kappa3,PEOE_VSA2,ERK2_inhibition,PKM2_inhibition
SMILES,,,,,,,,,,,,,,,,,,,,,
Nc1ccn([C@H]2CC[C@@H](CO)O2)c(=O)n1,403.834197,211.095691,10.129180,6,-2.272486,82,15,3.514917,15.577058,3.995325,...,22.747491,0,0,-0.024496,6.420822,1,2.000871,4.567100,1,0
O=[N+]([O-])c1ccccc1Nc1ccccc1,497.381764,214.074228,10.167785,4,-1.956483,80,16,3.265284,5.316789,4.439868,...,64.711622,2,2,-0.383476,5.687386,2,2.170475,10.114318,1,0
COc1cc([C@H]2CC(=O)c3c([nH]c(C(=O)OCCOC(C)C)c3C)C2)cc(OC)c1OC,967.469323,445.210052,23.658689,8,-2.185378,174,32,8.021039,28.668293,10.080000,...,40.210945,1,1,0.017220,53.517892,2,4.748179,4.794537,1,0
COc1ccc([C@@H]2CC(=O)c3cnc(NC(=O)c4ccccc4OC)nc3C2)c(OC)c1,1178.046320,433.163771,21.298075,8,-2.226078,164,32,7.344615,14.210589,9.050952,...,71.045696,2,2,0.096158,46.911010,3,4.115557,14.905863,1,0
Cn1c(C(=O)N2CCC[C@@H](C(=O)NCCCSC3CCCCC3)C2)cc2sccc21,869.049182,447.201419,21.750350,5,-2.455749,164,30,8.624397,14.783798,10.225943,...,23.206725,0,0,-0.125806,6.544756,2,5.047564,9.589074,1,0
CCCC(=O)Nc1nc2ccc(NC(=O)c3ccc(Br)o3)cc2s1,902.999838,406.993924,16.660232,6,-2.100403,124,24,5.057580,15.050728,7.144152,...,40.761598,1,1,-0.115751,21.909282,3,3.824473,9.589074,1,0
CCCC(=O)c1ccc(N2CCN(C(=O)c3ccccc3Br)CC2)c(F)c1,847.825294,432.084868,19.548795,4,-2.486819,142,27,6.598462,9.799819,8.652255,...,63.881413,2,2,0.074514,49.414137,2,4.378773,9.589074,1,0
CN1CC[NH+](CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1,831.885787,408.171580,19.609276,3,-3.143489,150,28,7.558517,9.799819,7.968146,...,48.028021,2,2,-0.903995,5.687386,2,4.161993,4.899910,1,0
COc1cc2c(cc1OC)-c1c/c(=N\c3c(C)cc(C)cc3C)n(C)c(=O)n1CC2,1259.069485,405.205242,20.011552,6,-2.399905,156,30,7.854472,9.473726,7.518920,...,68.557629,2,2,0.354092,23.531487,3,3.220199,9.134199,1,0
